In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('dataset/survey.csv')

In [3]:
train_df.shape

(1259, 27)

In [4]:
train_df = train_df.drop(train_df.loc[:, 'benefits':'comments'].columns, axis=1)
train_df = train_df.drop(['Timestamp','no_employees','work_interfere','state','remote_work'], axis= 1)
train_df

,Age,Gender,Country,self_employed,family_history,treatment,tech_company
0,37,Female,United States,NaN,No,Yes,Yes
1,44,M,United States,NaN,No,No,No
2,32,Male,Canada,NaN,No,No,Yes
3,31,Male,United Kingdom,NaN,Yes,Yes,Yes
4,31,Male,United States,NaN,No,No,Yes
...,...,...,...,...,...,...,...
1254,26,male,United Kingdom,No,No,Yes,Yes
1255,32,Male,United States,No,Yes,Yes,Yes
1256,34,male,United States,No,Yes,Yes,Yes
1257,46,f,United States,No,No,No,Yes


In [5]:
print(train_df.shape)

(1259, 8)


In [5]:
defaultInt = 0
defaultString = 'NaN'
defaultFloat = 0.0

# Create lists by data tpe
intFeatures = ['Age']
stringFeatures = ['Gender', 'Country', 'self_employed', 'family_history', 'treatment', 
                  'tech_company']
floatFeatures = []

# Clean the NaN's
for feature in train_df:
    if feature in intFeatures:
        train_df[feature] = train_df[feature].fillna(defaultInt)
    elif feature in stringFeatures:
        train_df[feature] = train_df[feature].fillna(defaultString)
    elif feature in floatFeatures:
        train_df[feature] = train_df[feature].fillna(defaultFloat)
    else:
        print('Error: Feature %s not recognized.' % feature)
train_df.head(5)   

,Age,Gender,Country,self_employed,family_history,treatment,tech_company
0,37,Female,United States,NaN,No,Yes,Yes
1,44,M,United States,NaN,No,No,No
2,32,Male,Canada,NaN,No,No,Yes
3,31,Male,United Kingdom,NaN,Yes,Yes,Yes
4,31,Male,United States,NaN,No,No,Yes


In [8]:
#clean 'Gender'
#Slower case all columm's elements
gender = train_df['Gender'].str.lower()
#print(gender)

#Select unique elements
gender = train_df['Gender'].unique()

#Made gender groups
male_str = ["male", "m", "male-ish", "maile", "mal", "male (cis)", "make", "male ", "man","msle", "mail", "malr","cis man", "Cis Male", "cis male"]
trans_str = ["trans-female", "something kinda male?", "queer/she/they", "non-binary","nah", "all", "enby", "fluid", "genderqueer", "androgyne", "agender", "male leaning androgynous", "guy (-ish) ^_^", "trans woman", "neuter", "female (trans)", "queer", "ostensibly male, unsure what that really means"]           
female_str = ["cis female", "f", "female", "woman",  "femake", "female ","cis-female/femme", "female (cis)", "femail"]

for (row, col) in train_df.iterrows():

    if str.lower(col.Gender) in male_str:
        train_df['Gender'].replace(to_replace=col.Gender, value='male', inplace=True)

    if str.lower(col.Gender) in female_str:
        train_df['Gender'].replace(to_replace=col.Gender, value='female', inplace=True)

    if str.lower(col.Gender) in trans_str:
        train_df['Gender'].replace(to_replace=col.Gender, value='trans', inplace=True)

#Get rid of miscellaneous words
stk_list = ['A little about you', 'p']
train_df = train_df[~train_df['Gender'].isin(stk_list)]

print(train_df['Gender'].unique())

['female' 'male' 'trans']


/Users/ronaldgohjingwei/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [15]:
#complete missing age with mean
train_df['Age'].fillna(train_df['Age'].median(), inplace = True)

# Fill with media() values < 18 and > 120
s = pd.Series(train_df['Age'])
s[s<18] = train_df['Age'].median()
train_df['Age'] = s
s = pd.Series(train_df['Age'])
s[s>120] = train_df['Age'].median()
train_df['Age'] = s

#Ranges of Age
train_df['age_range'] = pd.cut(train_df['Age'], [0,20,30,65,100], labels=["0-20", "21-30", "31-65", "66-100"], include_lowest=True)

In [16]:
#There are only 0.014% of self employed so let's change NaN to NOT self_employed
#Replace "NaN" string from defaultString
train_df['self_employed'] = train_df['self_employed'].replace([defaultString], 'No')
print(train_df['self_employed'].unique())

['No' 'Yes']


check missing data

In [17]:
#missing data
total = train_df.isnull().sum().sort_values(ascending=False)
percent = (train_df.isnull().sum()/train_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)
print(missing_data)

                Total  Percent
Age                 0      0.0
Gender              0      0.0
Country             0      0.0
self_employed       0      0.0
family_history      0      0.0
treatment           0      0.0
tech_company        0      0.0
age_range           0      0.0


In [18]:
train_df['treatment'] = train_df['treatment'].replace('Yes',1)
train_df['treatment'] = train_df['treatment'].replace('No',0)

In [19]:
train_df

,Age,Gender,Country,self_employed,family_history,treatment,tech_company,age_range
0,37,female,United States,No,No,1,Yes,31-65
1,44,male,United States,No,No,0,No,31-65
2,32,male,Canada,No,No,0,Yes,31-65
3,31,male,United Kingdom,No,Yes,1,Yes,31-65
4,31,male,United States,No,No,0,Yes,31-65
...,...,...,...,...,...,...,...,...
1254,26,male,United Kingdom,No,No,1,Yes,21-30
1255,32,male,United States,No,Yes,1,Yes,31-65
1256,34,male,United States,No,Yes,1,Yes,31-65
1257,46,female,United States,No,No,0,Yes,31-65


In [20]:
train_df.to_csv('mental_health_edited.csv',index=False)